## Объекты

In [12]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>

* 1 ```typecode``` - это атрибут класса, которым мы воспользуемся, когда будем преобразовывать экземпляры ```Vector2d``` в последовательности байтов и наоборот
* 2 Преобразование ```x``` и ```y``` в тип ```float``` в методе ```__init__``` позволяет на ранней стадии обнаружить оишбки, это полезно в случае, когда конструктор ```Vector2d``` вызывается с неподходящими аргументами
* 3 Наличие метода ```__iter__``` делает ```Vector2d``` итерируемым; именно благодаря ему работает распаковка (например: ```x, y = my_vector```). Мы реализуем его просто с помощью генераторного выражения, которое отдает компоненты поочередно.
* 4 Метод ```__repr__``` строит строку, интеполируя компоненты с помощью синаксиса ```{!r}``` для получения их представления, возвращаемого функции ```repr```; поскольку ```Vector2d``` - итерируемый объект, ```*self``` поставляет компоненты ```x``` и ```y``` функции ```format```
* 5 Из итерируемого объекта ```Vector2d``` легко построить кортеж для отображения в виде упорядоченной пары
* 6 Для генерации объекта типа ```bytes``` мы преобразуем ```typecode``` в ```bytes``` и конкатенируем
* 7 ... с объектом ```bytes```, полученным преобразованием массива, который построен путем обхода экземпляра
* 8 Для быстрого сравнения всех компонентов мы строим кортежи из операндов. Это работает, когда операнды являются экземплярами класса ```Vector2d```, но не без проблем
* 9 Модулем вектора называется длина гипотенузы прямоугольного треугольника с катетами ```x``` и ```y```
* 10 Метод ```__bool__``` вызывает ```abs(self)``` для вычисления модуля, а затем преобразует полученное значение в тип ```bool```, так что 0.0 преобразуется в ```False```, а любое число, отличное от нуля, - в ```True```

In [5]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y) # 1

3.0 4.0


In [7]:
x, y = v1 # 2
x, y

(3.0, 4.0)

In [8]:
v1 # 3

Vector2d(3.0, 4.0)

In [9]:
v1_clone = eval(repr(v1)) # 4
v1 == v1_clone # 5

True

In [10]:
print(v1) # 6

(3.0, 4.0)


In [13]:
ocerts = bytes(v1) # 7
ocerts

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [14]:
abs(v1) # 8

5.0

In [15]:
bool(v1), bool(Vector2d(0, 0)) #9

(True, False)

* 1 К компонентам ```Vector2d``` можно обращаться напрямую, как к атрибутам *(методом чтения нет)*
* 2 Объект ```Vector2d``` можно распоковать в кортеж переменных
* 3 ```repr``` для объекта имитирует исходный конструированияэкземпляра
* 4 Использование ```eval``` показывает, что результат ```repr``` для ```Vector2d``` - точное представление вызова конструктора
* 5 ```Vector2d``` поддерживает сравнение с помощью ```==```; это полезно для тестирования
* 6 ```print``` вызывает функцию ```str```, которая для ```Vector2d``` порождает упорядоченную пару
* 7 ```bytes``` используется методом ```__bytes__``` для получения двоичного представления
* 8 ```abs``` вызывает метод ```__abs__```, чтобы вернуть модуль вектора
* 9 ```Vector2d``` пользуется методом ```__bool__```, чтобы вернуть ```False``` для объекта ```Vector2d``` нулевой длины, и ```True``` в противном случае

## Альтернативный конструктор

In [18]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # <4>
        return cls(*memv)  # <5>

* 1 Метод класса снабжен декоратором ```classmethod```
* 2 Аргумент ```self``` отсутвует; вместо него в аргументе ```cls``` прередается сам класс
* 3 Читаем ```typecode``` из первого байта
* 4 Создаем объект ```memoryview``` из двоичной последовательности октетов и приводим его к типу ```typecode```
* 5 Распаковываем ```memoryview```, получившийся в результате приведения типа,и получаем пару аргументов, необходимых конструктору

## Декораторы ```classmethod``` и ```staticmethod```

In [19]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args # 1

    @staticmethod
    def statmeth(*args):
        return args # 2

Demo.klassmeth() # 3

(__main__.Demo,)

In [20]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [21]:
Demo.statmeth() # 4

()

In [22]:
Demo.statmeth('spam')

('spam',)

* 1 ```klassmeth``` просто возвращает все позиционные аргументы
* 2 ```statmeth```делает тоже самое
* 3 Вне зависимости от способа вызова ```Demo.klassmeth``` получает класс ```Demo``` в качестве первого аргумента
* 4 ```Demo.statmeth``` ведет себя, как обычная функция

## Форматирование при выводе